In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('dataset/IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.drop_duplicates(inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_5168\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [7]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\hp\AppData\Local\Temp\ipykernel_5168\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [8]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\hp\AppData\Local\Temp\ipykernel_5168\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [9]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\hp\AppData\Local\Temp\ipykernel_5168\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [10]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [11]:
import gensim

In [12]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [13]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [15]:
len(story)

105773

In [17]:
model = gensim.models.Word2Vec(window=10,min_count=2)

In [18]:
model.build_vocab(story)

In [19]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875453, 6212140)

In [20]:
len(model.wv.index_to_key)

31845

In [21]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [22]:
document_vector(df['review'].values[0])

array([-0.19842131,  0.4696381 ,  0.1464226 ,  0.21745904, -0.14646834,
       -0.63571906,  0.23682024,  0.9505459 , -0.3262931 , -0.3235152 ,
       -0.29111582, -0.5137831 ,  0.06205136,  0.14587256,  0.20104463,
       -0.07592639, -0.01624624, -0.31900185, -0.09411856, -0.5795793 ,
        0.04553343,  0.292698  ,  0.07664304, -0.29848093, -0.290557  ,
       -0.02000025, -0.3227828 , -0.03918373, -0.30897763,  0.0225875 ,
        0.2949583 , -0.01764273,  0.20407455, -0.23245457, -0.11483708,
        0.49137667,  0.12144089, -0.4176556 , -0.2578084 , -0.7343974 ,
        0.11386997, -0.27379838,  0.05375929, -0.07461816,  0.49635848,
       -0.12203024, -0.25656396, -0.03403258,  0.1365635 ,  0.35387436,
        0.06956244, -0.3818082 , -0.41040275, -0.07400509, -0.07261509,
        0.203899  ,  0.24364123, -0.01634261, -0.2726899 ,  0.09118007,
        0.08092742,  0.07759406,  0.0090733 , -0.10331094, -0.45999533,
        0.2967155 ,  0.03010197,  0.11080201, -0.34058535,  0.29

In [23]:
from tqdm import tqdm

In [24]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [19:53<00:00,  8.37it/s]


In [25]:
X = np.array(X)

In [26]:
X[0]

array([-0.19842131,  0.4696381 ,  0.1464226 ,  0.21745904, -0.14646834,
       -0.63571906,  0.23682024,  0.9505459 , -0.3262931 , -0.3235152 ,
       -0.29111582, -0.5137831 ,  0.06205136,  0.14587256,  0.20104463,
       -0.07592639, -0.01624624, -0.31900185, -0.09411856, -0.5795793 ,
        0.04553343,  0.292698  ,  0.07664304, -0.29848093, -0.290557  ,
       -0.02000025, -0.3227828 , -0.03918373, -0.30897763,  0.0225875 ,
        0.2949583 , -0.01764273,  0.20407455, -0.23245457, -0.11483708,
        0.49137667,  0.12144089, -0.4176556 , -0.2578084 , -0.7343974 ,
        0.11386997, -0.27379838,  0.05375929, -0.07461816,  0.49635848,
       -0.12203024, -0.25656396, -0.03403258,  0.1365635 ,  0.35387436,
        0.06956244, -0.3818082 , -0.41040275, -0.07400509, -0.07261509,
        0.203899  ,  0.24364123, -0.01634261, -0.2726899 ,  0.09118007,
        0.08092742,  0.07759406,  0.0090733 , -0.10331094, -0.45999533,
        0.2967155 ,  0.03010197,  0.11080201, -0.34058535,  0.29

In [27]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [28]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [31]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7751627441161743